In [1]:
%matplotlib inline

In [2]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import jieba
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Preprocessing
* 中文分词

In [3]:
SOS_token = 0
EOS_token = 1


#define the vocab class for each language
class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in jieba.cut(sentence, cut_all=False, HMM=True):  #jieba分词
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [4]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s) #NFD表示字符应该分解为多个组合字符表示
        if unicodedata.category(c) != 'Mn'         #去掉音调符号
    )

#trim, and remove non-Chinese characters

def normalizeString(s):
    import re
    s = unicodeToAscii(s.strip())
    s = re.sub("[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？?、~@#￥%……&*（）]+", "", s)
    return s

In [5]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')
    print(len(lines))

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    print('pairs: %d'% len(pairs))

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [6]:
MAX_LENGTH = 15


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [7]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])  #construct the input_lang vocab
        output_lang.addSentence(pair[1])  #construct the output_lang vocab
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('pt', 'sc', True)  #普通话-四川话
print(random.choice(pairs))

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache


Reading lines...
11265
pairs: 11265
Read 11265 sentence pairs
Trimmed to 11265 sentence pairs
Counting words...


Loading model cost 0.437 seconds.
Prefix dict has been built succesfully.


Counted words:
sc 6696
pt 7079
['在下一个转角右转', '在下一个转角右转']


## Define the encoder-decoder translation architecture

In [8]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, in_embed_dim): #input_size: input_lang vocab size
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, in_embed_dim) 
        self.gru = nn.GRU(in_embed_dim, hidden_size) 

    def forward(self, input, hidden):                    #input and h_0
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [9]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, out_embed_dim, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.out_embed_dim = out_embed_dim
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.out_embed_dim) 
        self.attn = nn.Linear(self.hidden_size + self.out_embed_dim, self.max_length) 
        self.attn_combine = nn.Linear(self.hidden_size + self.out_embed_dim, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size) #output_size: output_lang vocab size

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)
        #why use 0 ? embedded[0], hidden[0]..., because it's the init state
        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0)) #weighted_sum

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

## Define the train, evaluate and helper function

In [10]:
def indexesFromSentence(lang, sentence):
    #lang: language vocab object
    return [lang.word2index[word] for word in jieba.cut(sentence)]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)  #append the EOS_token, it's crucial
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [11]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device) #初始化为全0

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0] # encoder_output[0, 0] means  encoder_output[0][0]

    decoder_input = torch.tensor([[SOS_token]], device=device) #1*1

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [12]:
import time

def asMinutes(s):
    m = s // 60
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [13]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)] #迭代n_iters次，每次随机从pairs中选一对
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion) #training
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [14]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [15]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device) #初始化为全0

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0] #注意是累加，而train中是赋值

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)    #初始化attention为全0

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')  #解码时遇到EOS_token，要将其加入到decoded_words,而在train中编码时，直接break
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()]) #加入解码的词

            decoder_input = topi.squeeze().detach()  #next decoder input

        return decoded_words, decoder_attentions[:di + 1] #返回attentions可用来做可视化

In [16]:
def evaluateRandomly(encoder, decoder, n=5):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1]) 
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ''.join(output_words) #机器翻译,中文以空字符连接，英文以空格连接
        print('<', output_sentence)
        print('')

## Training

In [17]:
hidden_size = 256    #rnn's hidden_size
in_embed_dim = 256   #input language word embedding dimension
out_embed_dim = 256  #output language word embedding dimension
encoder1 = EncoderRNN(input_lang.n_words, hidden_size, in_embed_dim).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, out_embed_dim, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 100000, print_every=1000, learning_rate=0.01)

#因为每次迭代都是随机选一对句子，而且使用了teacher forcing，所以波动会较大

0m 17s (- 28m 4s) (1000 1%) 5.0954
0m 24s (- 19m 48s) (2000 2%) 4.9560
0m 31s (- 17m 4s) (3000 3%) 4.7455
0m 39s (- 15m 44s) (4000 4%) 4.4024
0m 46s (- 14m 50s) (5000 5%) 4.2240
0m 54s (- 14m 12s) (6000 6%) 4.0779
1m 1s (- 13m 43s) (7000 7%) 3.9015
1m 9s (- 13m 21s) (8000 8%) 3.8624
1m 17s (- 13m 1s) (9000 9%) 3.6885
1m 25s (- 12m 45s) (10000 10%) 3.6106
1m 32s (- 12m 31s) (11000 11%) 3.5253
1m 40s (- 12m 16s) (12000 12%) 3.3539
1m 48s (- 12m 4s) (13000 13%) 3.1009
1m 56s (- 11m 52s) (14000 14%) 3.1515
2m 3s (- 11m 42s) (15000 15%) 3.0632
2m 11s (- 11m 32s) (16000 16%) 2.9201
2m 19s (- 11m 21s) (17000 17%) 2.8737
2m 27s (- 11m 11s) (18000 18%) 2.7938
2m 35s (- 11m 2s) (19000 19%) 2.6918
2m 43s (- 10m 52s) (20000 20%) 2.5874
2m 50s (- 10m 42s) (21000 21%) 2.5747
2m 58s (- 10m 33s) (22000 22%) 2.4601
3m 6s (- 10m 24s) (23000 23%) 2.5331
3m 14s (- 10m 16s) (24000 24%) 2.3646
3m 22s (- 10m 7s) (25000 25%) 2.1649
3m 30s (- 9m 58s) (26000 26%) 2.2796
3m 38s (- 9m 49s) (27000 27%) 2.2615
3m 4

In [18]:
evaluateRandomly(encoder1, attn_decoder1)

> 汤姆住在波士顿吗
= 汤姆住在波士顿吗
< 汤姆住在波士顿吗<EOS>

> 汤姆认为我太年轻
= 汤姆认为我太年轻
< 汤姆认为我太太<EOS>

> 这是你的书
= 这是你的书
< 这是你的书<EOS>

> 你为甚么不锁门
= 你为甚么不锁门
< 你为甚么不锁门<EOS>

> 他站在门后
= 他站在门后
< 他站在后后<EOS>



In [19]:
def evaluateInput(encoder, decoder):
    input_sentence = ''
    print('天不怕地不怕，就怕普通人讲四川话')
    while(1):
        try:
            # Get input sentence
            input_sentence = input('普通话> ')
            # Check if it is quit case
            if input_sentence == 'q' or input_sentence == 'quit': break
            # Normalize sentence
            input_sentence = normalizeString(input_sentence)
            # Evaluate sentence
            output_words = evaluate(encoder, decoder, input_sentence)[0]
            print('四川话:', ''.join(output_words[:-1]))

        except KeyError:
            print("Error: Encountered unknown word.")

In [21]:
evaluateInput(encoder1, attn_decoder1)

天不怕地不怕，就怕普通人讲四川话
普通话> 你想做什么
四川话: 你想做啥子
普通话> 我要吃肉
四川话: 我要吃肉
普通话> 你怎么这么那个
四川话: 你啷个这么那个
普通话> 你动动我试试
四川话: 你傍哈我告哈耶
普通话> 我明天不上班
四川话: 老子明天不上班
普通话> q


### 求语料
* 普通话-四川话平行语料，多多益善！